In [ ]:

!pip install kagglehub --quiet


In [ ]:
import kagglehub
paramaggarwal_fashion_product_images_dataset_path = kagglehub.dataset_download('paramaggarwal/fashion-product-images-dataset')
paramaggarwal_fashion_product_images_small_path = kagglehub.dataset_download('paramaggarwal/fashion-product-images-small')


100%|██████████| 23.1G/23.1G [03:41<00:00, 112MB/s]

Extracting files...


100%|██████████| 565M/565M [00:10<00:00, 58.2MB/s]

Extracting files...


In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.colors




dataset_path = '/kaggle/input/fashion-product-images-dataset/fashion-dataset/styles.csv'
data = pd.read_csv(dataset_path, on_bad_lines='skip')

print("Dataset Overview:")
print(data.head())

columns_to_keep = ['id', 'gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'usage']
data = data[columns_to_keep]

data.dropna(subset=['gender', 'masterCategory', 'baseColour'], inplace=True)
data['baseColour'] = data['baseColour'].fillna('Unknown')

data.columns = [col.strip().lower() for col in data.columns]

print("\nUnique values in gender:", data['gender'].unique())
print("Unique values in season:", data['season'].unique())
print("Unique values in usage:", data['usage'].unique())


In [ ]:

def display_box(face, width=200, height=100):
    display(HTML(f'<div style="width:{width}px; height:{height}px; background-color:{face[0]}; border: 2px solid black;">{face[1]}</div>'))


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
import PIL.Image
import io

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
          const div = document.createElement('div');
          const capture = document.createElement('button');
          capture.textContent = '📸 Capture';
          div.appendChild(capture);

          const video = document.createElement('video');
          video.style.display = 'block';
          const stream = await navigator.mediaDevices.getUserMedia({video: true});

          document.body.appendChild(div);
          div.appendChild(video);
          video.srcObject = stream;
          await video.play();

          // Resize the output to fit the video stream.
          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

          // Wait for Capture to be clicked.
          await new Promise((resolve) => capture.onclick = resolve);

          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);
          stream.getTracks().forEach(track => track.stop());
          div.remove();

          const dataUrl = canvas.toDataURL('image/jpeg', quality);
          return dataUrl;
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
from PIL import ImageColor
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# Skin tones dictionary
skin_tones = {
    "#373028": "Deepest Skin",
    "#422811": "Very Deep",
    "#513B2E": "Deep Brown",
    "#6F503C": "Medium Brown",
    "#81654F": "Tan",
    "#9D7A54": "Light Tan",
    "#BEA07E": "Medium Fair",
    "#E5C8A6": "Light Fair",
    "#E7C1B8": "Warm Fair",
    "#F3DAD6": "Very Fair",
    "#FBF2F3": "Pale",
}

# Skin tone detection function
def detect_skin_tone(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized_image = cv2.resize(image, (200, 200))
    avg_color = resized_image.mean(axis=0).mean(axis=0)
    avg_color_hex = "#{:02x}{:02x}{:02x}".format(int(avg_color[0]), int(avg_color[1]), int(avg_color[2]))
    avg_color_rgb = np.array(ImageColor.getrgb(avg_color_hex))

    closest_tone_hex = min(
        skin_tones.keys(),
        key=lambda hex_code: np.linalg.norm(avg_color_rgb - np.array(ImageColor.getrgb(hex_code)))
    )
    return closest_tone_hex, skin_tones[closest_tone_hex]

# Face cropping and detection class
class FaceCropper:
    def __init__(self):
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    def generate(self, image_path, show_result=True):
        img = cv2.imread(image_path)
        if img is None:
            print("❌ Could not open image.")
            return None, None

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

        if len(faces) == 0:
            print("❌ No faces detected.")
            return None, None

        if show_result:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            cropped_faces = []

            for (x, y, w, h) in faces:
                face_img = img[y:y+h, x:x+w]
                resized_face = cv2.resize(face_img, (200, 200))
                cropped_faces.append((resized_face, (x, y, w, h)))
                cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (255, 0, 0), 2)

            fig, ax = plt.subplots(1, 3, figsize=(16, 6))
            ax[0].imshow(img_rgb)
            ax[0].axis('off')
            ax[0].set_title('Original with Face Detection')

            zoomed_face, _ = cropped_faces[0]
            zoomed_face_rgb = cv2.cvtColor(zoomed_face, cv2.COLOR_BGR2RGB)
            ax[1].imshow(zoomed_face_rgb)
            ax[1].axis('off')
            ax[1].set_title('Cropped Face')

            skin_hex, tone_name = detect_skin_tone(zoomed_face)
            tone_rgb = matplotlib.colors.hex2color(skin_hex)
            tone_block = np.ones((10, 10, 3)) * tone_rgb
            ax[2].imshow(tone_block)
            ax[2].axis('off')
            ax[2].set_title(f"Skin Tone: {tone_name} ({skin_hex})")

            plt.show()
        else:
            # If not showing result, still detect skin tone on first face
            zoomed_face = img[faces[0][1]:faces[0][1]+faces[0][3], faces[0][0]:faces[0][0]+faces[0][2]]
            zoomed_face = cv2.resize(zoomed_face, (200, 200))
            skin_hex, tone_name = detect_skin_tone(zoomed_face)

        # Save faces as 32x32 images
        for i, (x, y, w, h) in enumerate(faces, start=1):
            face_img = img[y:y + h, x:x + w]
            resized = cv2.resize(face_img, (32, 32))
            cv2.imwrite(f"image{i}.jpg", resized)
        print(f"✅ {len(faces)} face(s) detected and saved.")

        return skin_hex, tone_name

# Webcam capture function for Google Colab
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = '📸 Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getTracks().forEach(track => track.stop());
      div.remove();

      const dataUrl = canvas.toDataURL('image/jpeg', quality);
      return dataUrl;
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# User input for image source
choice = input("📸 Enter 'w' for webcam or 'u' to upload an image: ").strip().lower()

if choice == 'w':
    print("📷 Capturing from webcam...")
    image_path = take_photo()
elif choice == 'u':
    from google.colab import files
    uploaded = files.upload()
    image_path = list(uploaded.keys())[0]
else:
    print("❌ Invalid option.")
    image_path = None

if image_path:
    detector = FaceCropper()
    detected_tone_hex, detected_tone_name = detector.generate(image_path)

    if detected_tone_hex is not None:
        skin_tone_to_color_mapping = {
            "#373028": ["Navy Blue", "Black", "Charcoal", "Burgundy", "Maroon", "Olive", "Rust", "Gold", "Cream", "Peach"],
            "#422811": ["Navy Blue", "Brown", "Khaki", "Olive", "Maroon", "Mustard", "Teal", "Tan", "Rust", "Burgundy"],
            "#513B2E": ["Cream", "Beige", "Olive", "Burgundy", "Red", "Orange", "Mustard", "Bronze", "Teal", "Peach"],
            "#6F503C": ["Beige", "Brown", "Green", "Khaki", "Cream", "Peach", "Lime Green", "Olive", "Maroon", "Rust", "Mustard"],
            "#81654F": ["Beige", "Off White", "Sea Green", "Cream", "Lavender", "Mauve", "Burgundy", "Yellow", "Lime Green"],
            "#9D7A54": ["Olive", "Khaki", "Yellow", "Sea Green", "Turquoise Blue", "Coral", "White", "Gold", "Peach"],
            "#BEA07E": ["Coral", "Sea Green", "Turquoise Blue", "Pink", "Lavender", "Rose", "White", "Peach", "Teal", "Fluorescent Green"],
            "#E5C8A6": ["Turquoise Blue", "Peach", "Teal", "Pink", "Red", "Rose", "Off White", "White", "Cream", "Gold", "Yellow"],
            "#E7C1B8": ["Pink", "Rose", "Peach", "White", "Off White", "Beige", "Lavender", "Teal", "Fluorescent Green"],
            "#F3DAD6": ["White", "Cream", "Peach", "Pink", "Rose", "Lavender", "Mustard", "Lime Green", "Light Blue", "Fluorescent Green"],
            "#FBF2F3": ["Soft Pastels (Peach, Lavender, Pink)", "White", "Off White", "Rose", "Light Blue", "Sea Green", "Fluorescent Green", "Silver", "Cream", "Tan"]
        }

        recommended_colors = skin_tone_to_color_mapping.get(detected_tone_hex, [])

        print(f"Recommended Colors for {detected_tone_hex} ({detected_tone_name}): {recommended_colors}")
    else:
        print("Could not detect skin tone.")


In [ ]:
import pandas as pd
import random

user_gender = input("Please enter your gender (Men/Women): ").strip().capitalize()

if user_gender not in ['Men', 'Women']:
    print("Invalid input. Defaulting to 'Women'.")
    user_gender = 'Women'

filtered_data = data[data['gender'] == user_gender]


gender_filter = 'Women'
user_usage = ["Formal"]
footwear_type = "Casual"
recommended_colors.extend(["Black", "White", "Beige", "Cream", "Off White", "Grey", "Charcoal"])

filtered_data = filtered_data[filtered_data['basecolour'].isin(recommended_colors)]

top_wear = filtered_data[filtered_data['subcategory'] == 'Topwear']
top_wear = top_wear[top_wear['usage'].isin(user_usage)]

bottom_wear = filtered_data[filtered_data['subcategory'] == 'Bottomwear']
bottom_wear = bottom_wear[bottom_wear['usage'].isin(user_usage)]

footwear = filtered_data[filtered_data['mastercategory'] == 'Footwear']

if "Formal" in user_usage:
    footwear = footwear[footwear['subcategory'].isin(['Shoes'])]

print(f"Filtered items - Topwear: {len(top_wear)}, Bottomwear: {len(bottom_wear)}, Footwear: {len(footwear)}")

def get_complementary(color, palette):
    """Return a complementary color if available in the palette."""
    return random.choice([c for c in palette if c != color])

def get_analogous(color, palette):
    """Return an analogous color if available in the palette."""
    return random.choice([c for c in palette if c != color])

def get_neutral(palette):
    """Return a neutral color from the palette."""
    neutrals = ["Black", "White", "Beige", "Cream", "Off White", "Grey", "Charcoal"]
    return random.choice([c for c in palette if c in neutrals])

outfit_combinations = []
for top in top_wear.head(10).itertuples():
    for bottom in bottom_wear.head(100).itertuples():
        for foot in footwear.head(100).itertuples():
            bottom_color = get_complementary(top.basecolour, recommended_colors)
            footwear_color = random.choice([get_analogous(bottom_color, recommended_colors), get_neutral(recommended_colors)])

            outfit_combinations.append({
                "Topwear": top.id,
                "Bottomwear": bottom.id,
                "Footwear": foot.id,
                "Topwear Color": top.basecolour,
                "Bottomwear Color": bottom_color,
                "Footwear Color": footwear_color
            })



images_links = pd.read_csv("/kaggle/input/fashion-product-images-dataset/fashion-dataset/images.csv", on_bad_lines='skip')
random_combinations = random.sample(outfit_combinations, min(len(outfit_combinations), 5))
from IPython.display import display, HTML, Image

def display_combination_images(top_id, bottom_id, foot_id):
    top_img = list(images_links[images_links['filename'] == str(top_id) + ".jpg"].link)[0]
    bottom_img = list(images_links[images_links['filename'] == str(bottom_id) + ".jpg"].link)[0]
    foot_img = list(images_links[images_links['filename'] == str(foot_id) + ".jpg"].link)[0]


    html_code = f"""
    <div style="display: flex; justify-content: space-evenly; align-items: center;">
        <div style="text-align: center;">
            <h3>Topwear</h3>
            <img src="{top_img}" width="200px" />
        </div>
        <div style="text-align: center;">
            <h3>Bottomwear</h3>
            <img src="{bottom_img}" width="200px" />
        </div>
        <div style="text-align: center;">
            <h3>Footwear</h3>
            <img src="{foot_img}" width="200px" />
        </div>
    </div>
    """

    display(HTML(html_code))




for i, combo in enumerate(random_combinations):
    print(f"Outfit {i + 1}")
    display_combination_images(combo['Topwear'],combo['Bottomwear'],combo['Footwear'])


Please enter your gender (Men/Women): Women
Filtered items - Topwear: 15, Bottomwear: 37, Footwear: 1674
Outfit 1


Outfit 2


Outfit 3


Outfit 4


Outfit 5
